# Adding a custom HFC to FaIR

This assumes that we want to run an existing scenario (say ssp245) and add a HFC species - let's say HFC-152 - that is not included in the default scenario set.

We can also do pulse experiments, which would be quite a bit easier - remove all of the other things.

This notebook is based on the script that runs calibrated, constrained projections from the SSPs that is developed as part of the `fair-calibrate` package. In particular, we use v1.1.0 of the calibration, that corrects the NOx emissions.

## Modifying species files

For ease, the `specie`s are defined in CSV files called `species_config_properties_*.csv` in the `data/` directory of the repository. These contain all of the species and properties definitions which are read in. It is much easier to construct a CSV this way and override any default values that are introduced as part of the Monte Carlo ensemble than to define everything in the notebook. 

The only difference between `with` and `without` is that `with` includes HFC-152, including its lifetime, molecular weight and radiative efficiency. For clarity it is the last line of the CSV.

In this row:
- name: `HFC-152`
- type: `f-gas`. There are many reserved types; an `f-gas` is a greenhouse gas that does not contribute to ozone depletion (which is either `cfc-11` or `other-halogen`) and is not `co2`, `ch4`, or `n2o`
- input_mode: `emissions` (can be `emissions`, `concentration` or `forcing`, to specify how you want to drive the climate simulation). Some species allow `calculated` in the sense that forcings will be calculated from precursors of other species; this does not apply to type `f-gas` and you would get an error
- greenhouse_gas: 1 (boolean value). It's a greenhouse gas, so tell FaIR to treat as such.
- aerosol_chemistry_from_emissions: 0 (boolean value). Emissions of HFC-152 are not assumed to affect aerosol, ozone or methane lifetime.
- aerosol_chemistry_from_concentration: 0 (boolean value). HFCs are assumed to be inert.
- partition_fraction\[0-3\]: From v2.0.0 onwards FaIR uses an n-box model (default n=4) for the atmospheric concentration of GHGs. Only CO2 is assumed to have multiple lifetimes - for everything else we tend to set \[1, 0, 0, 0\]. Note, the partition fractions must sum to one.
- unperturbed_lifetime\[0-3\]: 0.471 years from AR6 WG1 Ch7. The lifetime of GHGs in each box in the absence of feedbacks. For non-CO2 GHGs, the last three values don't matter because the partition fraction is zero.
- tropospheric_adjustment: any radiative adjustment to go from radiative forcing to effective radiative forcing (see Smith et al. 2018; 2020; Hodnebrog et al. 2020). For most species this is zero (or known without confidence, so assumed zero).
- forcing_efficacy: the temperature change per W/m2 of this agent as a function of what would be expected due to the climate feedback parameter. For almost all species this is one, or there is not sufficient evidence to say that it isn't. Volcanic is set to 0.6, as surface temperature records do not predict as large a response as would be expected from its forcing.
- forcing_temperature_feedback: does surface temperature affect the forcing? Values given in W/m2/K. For ozone it does (chemical effect), for methane it does but through its lifetime rather than its radiative efficiency, so this is not changed. For all other compounds, assumed zero.
- forcing_scale: a multiplicative factor linking the best estimate forcing generated by FaIR to a pre-defined scale factor. Usually used to sample uncertainty in forcing strengths assessed by IPCC, and overridden on a per-config basis. So it is set to 1 by default.
- molecular_weight: 66.05 g/mol (source: PubChem)
- baseline_concentration: 0 ppt (assumed) - the pre-industrial concentration.
- forcing_reference_concentration: some equations need a baseline value for calculating forcings. For minor GHGs this should be the same as the `baseline_concentration`.
- forcing_reference_emissions: should be zero for GHGs
- iirf*: we don't know this in advance - will calculate and fill in
- baseline_emissions: assumed zero. What are pre-industrial, possibly natural, emissions. We need to subtract these from any emissions time series when calculating atmospheric concentration increases (for GHGs), or forcing (for SLCFs).
- g\[0,1\]: will be calculated
- greenhouse_gas_radiative_efficiency: 0.045 W/m2/ppb, source AR6 WG1 Ch7
- contrails_radiative_efficiency: 0 (assume HFC152 is not involved in contrail formation)
- erfari_radiative_efficiency: 0 (assume HFC152 does not contribute to direct aerosol formation)
- h2o_stratospheric_factor: 0 (does not contribute to stratospheric water formation)
- lapsi_radiative_efficiency: 0 (does not contribute to snow albedo darkening)
- land_use_cumulative_emissions_to_forcing: 0 (does not contribute to land use change)
- ozone_radiative_efficiency: 0 W/m2/ppt (does not contribute to ozone formation)
- cl_atoms: 0 (no chlorine atoms)
- br_atoms: 0 (no bromine atoms)
- fractional_release: 0 (related to ozone depletion, and not relevant)
- aci_scale: nan (only relevant to aerosol-cloud interactions)
- aci_shape: 0 (does not contribute to indirect aerosol forcing)
- ch4_lifetime_chemical_sensitivity: 0 (does not affect methane lifetime)
- lifetime_temperature_sensitivity: nan (only relevant for methane)

In [ ]:
import os

import matplotlib.pyplot as pl
import numpy as np
import pandas as pd
import pooch

from fair import FAIR
from fair.interface import fill, initialise
from fair.io import read_properties

As we're running scenarios that have different numbers of species, we cannot parallelise this across the scenario dimension, so we have to create two different instances of the `FAIR()` class.

In [ ]:
scenarios_without = ['ssp245']
scenarios_with = ['ssp245_hfc152']

## External forcings

Solar and volcanic forcings are loaded in

In [ ]:
df_solar = pd.read_csv(
    "../data/solar_erf_timebounds.csv", index_col="year"
)
df_volcanic = pd.read_csv(
    "../data/volcanic_ERF_monthly_174901-201912.csv"
)

In [ ]:
solar_forcing = np.zeros(551)
volcanic_forcing = np.zeros(551)
for i, year in enumerate(np.arange(1750, 2021)):
    volcanic_forcing[i] = np.mean(
        df_volcanic.loc[
            ((year - 1) <= df_volcanic["year"]) & (df_volcanic["year"] < year)
        ].erf
    )
volcanic_forcing[271:281] = np.linspace(1, 0, 10) * volcanic_forcing[270]
solar_forcing = df_solar["erf"].loc[1750:2300].values

trend_shape = np.ones(551)
trend_shape[:271] = np.linspace(0, 1, 271)

## Grab the configs

Using `pooch` saves this to the cache, downloading directly from Zenodo, so does not appear as part of the Git repository.

In [ ]:
fair_params_1_1_0_obj = pooch.retrieve(
    url = 'https://zenodo.org/record/7694879/files/calibrated_constrained_parameters.csv',
    known_hash = 'md5:9f236c43dd18a36b7b63b94e05f3caab',
)

In [ ]:
df_configs = pd.read_csv(fair_params_1_1_0_obj, index_col=0)
valid_all = df_configs.index

In [ ]:
df_configs.loc[223, 'r0':'rA']

## Run FaIR without extra HFC

We'll call this instance `f_without`. The `read_properties()` line points to the CSV file that does not include HFC-152 (a slightly modified default).

In [ ]:
f_without = FAIR(ch4_method="Thornhill2021")
f_without.define_time(1750, 2300, 1)
f_without.define_scenarios(scenarios_without)
f_without.define_configs(valid_all)
species_without, properties_without = read_properties(filename='../data/species_configs_properties_without.csv')
f_without.define_species(species_without, properties_without)
f_without.allocate()

### Put in emissions from ssp245

Usually we'd use `fill_from_rcmip()` to do this, but since we're slightly customising here (and the NOx emissions are wrong) we'll do it the long-winded way: download the RCMIP file, correct the NOx, and fill in the `emissions` array.

In [ ]:
rcmip_emissions_file = pooch.retrieve(
    url="doi:10.5281/zenodo.4589756/rcmip-emissions-annual-means-v5-1-0.csv",
    known_hash="md5:4044106f55ca65b094670e7577eaf9b3",
)

In [ ]:
df_emis = pd.read_csv(rcmip_emissions_file)

This next cell corrects the NOx and fills in the NOx emissions in the input emissions array. What's the issue? The GFED emissions relating to biomass burning have units of NO, whereas the CEDS emissions relating to fossil, industrial and agriculture have units of NO2. The conversion was not made in the RCMIP data. FaIR uses NO2 consistently.

In [ ]:
gfed_sectors = [
    "Emissions|NOx|MAGICC AFOLU|Agricultural Waste Burning",
    "Emissions|NOx|MAGICC AFOLU|Forest Burning",
    "Emissions|NOx|MAGICC AFOLU|Grassland Burning",
    "Emissions|NOx|MAGICC AFOLU|Peat Burning",
]

for scenario in scenarios_without:
    f_without.emissions.loc[dict(specie="NOx", scenario=scenario)] = (
        df_emis.loc[
            (df_emis["Scenario"] == scenario)
            & (df_emis["Region"] == "World")
            & (df_emis["Variable"].isin(gfed_sectors)),
            "1750":"2500",
        ]
        .interpolate(axis=1)
        .values.squeeze()
        .sum(axis=0)
        * 46.006
        / 30.006
        + df_emis.loc[
            (df_emis["Scenario"] == scenario)
            & (df_emis["Region"] == "World")
            & (df_emis["Variable"] == "Emissions|NOx|MAGICC AFOLU|Agriculture"),
            "1750":"2500",
        ]
        .interpolate(axis=1)
        .values.squeeze()
        + df_emis.loc[
            (df_emis["Scenario"] == scenario)
            & (df_emis["Region"] == "World")
            & (df_emis["Variable"] == "Emissions|NOx|MAGICC Fossil and Industrial"),
            "1750":"2500",
        ]
        .interpolate(axis=1)
        .values.squeeze()
    )[:550][:, None]

In [ ]:
# from FaIR name to RCMIP name. NOx is excluded from this list as we don't want to pick up the uncorrected value.
# aviation NOx is a CEDS product so it's fine
species_mapping = {'BC': 'Emissions|BC',
 'CCl4': 'Emissions|Montreal Gases|CCl4',
 'CFC-11': 'Emissions|Montreal Gases|CFC|CFC11',
 'CFC-113': 'Emissions|Montreal Gases|CFC|CFC113',
 'CFC-114': 'Emissions|Montreal Gases|CFC|CFC114',
 'CFC-115': 'Emissions|Montreal Gases|CFC|CFC115',
 'CFC-12': 'Emissions|Montreal Gases|CFC|CFC12',
 'CH2Cl2': 'Emissions|Montreal Gases|CH2Cl2',
 'CH3Br': 'Emissions|Montreal Gases|CH3Br',
 'CH3CCl3': 'Emissions|Montreal Gases|CH3CCl3',
 'CH3Cl': 'Emissions|Montreal Gases|CH3Cl',
 'CH4': 'Emissions|CH4',
 'CHCl3': 'Emissions|Montreal Gases|CHCl3',
 'CO': 'Emissions|CO',
 'CO2 AFOLU': 'Emissions|CO2|MAGICC AFOLU',
 'CO2 FFI': 'Emissions|CO2|MAGICC Fossil and Industrial',
 'HCFC-141b': 'Emissions|Montreal Gases|HCFC141b',
 'HCFC-142b': 'Emissions|Montreal Gases|HCFC142b',
 'HCFC-22': 'Emissions|Montreal Gases|HCFC22',
 'HFC-125': 'Emissions|F-Gases|HFC|HFC125',
 'HFC-134a': 'Emissions|F-Gases|HFC|HFC134a',
 'HFC-143a': 'Emissions|F-Gases|HFC|HFC143a',
 'HFC-152a': 'Emissions|F-Gases|HFC|HFC152a',
 'HFC-227ea': 'Emissions|F-Gases|HFC|HFC227ea',
 'HFC-23': 'Emissions|F-Gases|HFC|HFC23',
 'HFC-236fa': 'Emissions|F-Gases|HFC|HFC236fa',
 'HFC-245fa': 'Emissions|F-Gases|HFC|HFC245fa',
 'HFC-32': 'Emissions|F-Gases|HFC|HFC32',
 'HFC-365mfc': 'Emissions|F-Gases|HFC|HFC365mfc',
 'HFC-4310mee': 'Emissions|F-Gases|HFC|HFC4310mee',
 'Halon-1202': 'Emissions|Montreal Gases|Halon1202',
 'Halon-1211': 'Emissions|Montreal Gases|Halon1211',
 'Halon-1301': 'Emissions|Montreal Gases|Halon1301',
 'Halon-2402': 'Emissions|Montreal Gases|Halon2402',
 'N2O': 'Emissions|N2O',
 'NF3': 'Emissions|F-Gases|NF3',
 'NH3': 'Emissions|NH3',
 'NOx aviation': 'Emissions|NOx|MAGICC Fossil and Industrial|Aircraft',
 'OC': 'Emissions|OC',
 'C2F6': 'Emissions|F-Gases|PFC|C2F6',
 'C3F8': 'Emissions|F-Gases|PFC|C3F8',
 'C4F10': 'Emissions|F-Gases|PFC|C4F10',
 'C5F12': 'Emissions|F-Gases|PFC|C5F12',
 'C6F14': 'Emissions|F-Gases|PFC|C6F14',
 'C7F16': 'Emissions|F-Gases|PFC|C7F16',
 'C8F18': 'Emissions|F-Gases|PFC|C8F18',
 'CF4': 'Emissions|F-Gases|PFC|CF4',
 'c-C4F8': 'Emissions|F-Gases|PFC|cC4F8',
 'SF6': 'Emissions|F-Gases|SF6',
 'SO2F2': 'Emissions|F-Gases|SO2F2',
 'Sulfur': 'Emissions|Sulfur',
 'VOC': 'Emissions|VOC'}

In [ ]:
# change units for some emitted species
unit_convert = {specie: 1 for specie in species_without}
unit_convert['CO2 AFOLU'] = 0.001
unit_convert['CO2 FFI'] = 0.001
unit_convert['N2O'] = 0.001

Now fill in the other emitted species.

In [ ]:
for scenario in scenarios_without:
    for specie in species_mapping:
        f_without.emissions.loc[dict(specie=specie, scenario=scenario)] = (
            df_emis.loc[
                (df_emis["Scenario"] == scenario)
                & (df_emis["Region"] == "World")
                & (df_emis["Variable"] == species_mapping[specie]),
                "1750":"2500",
            ]
            .interpolate(axis=1)
            .values.squeeze()
            * unit_convert[specie]
        )[:550][:, None]

In [ ]:
f_without.emissions

### Put in solar and volcanic forcing

In [ ]:
fill(
    f_without.forcing,
    volcanic_forcing[:, None, None] * df_configs["scale Volcanic"].values.squeeze(),
    specie="Volcanic",
)
fill(
    f_without.forcing,
    solar_forcing[:, None, None] * df_configs["solar_amplitude"].values.squeeze()
    + trend_shape[:, None, None] * df_configs["solar_trend"].values.squeeze(),
    specie="Solar",
)

### Monte Carlo climate configs

In [ ]:
fill(f_without.climate_configs["ocean_heat_capacity"], df_configs.loc[:, "c1":"c3"].values)
fill(
    f_without.climate_configs["ocean_heat_transfer"],
    df_configs.loc[:, "kappa1":"kappa3"].values,
)
fill(f_without.climate_configs["deep_ocean_efficacy"], df_configs["epsilon"].values.squeeze())
fill(f_without.climate_configs["gamma_autocorrelation"], df_configs["gamma"].values.squeeze())
fill(f_without.climate_configs["sigma_eta"], df_configs["sigma_eta"].values.squeeze())
fill(f_without.climate_configs["sigma_xi"], df_configs["sigma_xi"].values.squeeze())
fill(f_without.climate_configs["seed"], df_configs["seed"])
fill(f_without.climate_configs["stochastic_run"], True)
fill(f_without.climate_configs["use_seed"], True)
fill(f_without.climate_configs["forcing_4co2"], df_configs["F_4xCO2"])

### Monte Carlo species configs

In the first line we need to point to the filename providing the configs. After this, we manually overrides what we loaded up in the CSV.

In [ ]:
f_without.fill_species_configs('../data/species_configs_properties_without.csv')

# carbon cycle
fill(f_without.species_configs["iirf_0"], df_configs["r0"].values.squeeze(), specie="CO2")
fill(f_without.species_configs["iirf_airborne"], df_configs["rA"].values.squeeze(), specie="CO2")
fill(f_without.species_configs["iirf_uptake"], df_configs["rU"].values.squeeze(), specie="CO2")
fill(f_without.species_configs["iirf_temperature"], df_configs["rT"].values.squeeze(), specie="CO2")

# aerosol indirect
fill(f_without.species_configs["aci_scale"], df_configs["beta"].values.squeeze())
fill(f_without.species_configs["aci_shape"], df_configs["shape Sulfur"].values.squeeze(), specie="Sulfur")
fill(f_without.species_configs["aci_shape"], df_configs["shape BC"].values.squeeze(), specie="BC")
fill(f_without.species_configs["aci_shape"], df_configs["shape OC"].values.squeeze(), specie="OC")

# aerosol direct
for specie in ["BC", "CH4", "N2O", "NH3", "NOx", "OC", "Sulfur", "VOC", "Equivalent effective stratospheric chlorine"]:
    fill(f_without.species_configs["erfari_radiative_efficiency"], df_configs[f"ari {specie}"], specie=specie)

# forcing scaling
for specie in ["CO2", "CH4", "N2O", "Stratospheric water vapour", "Contrails", "Light absorbing particles on snow and ice", "Land use"]:
    fill(f_without.species_configs["forcing_scale"], df_configs[f"scale {specie}"].values.squeeze(), specie=specie)

for specie in [
    "CFC-11",
    "CFC-12",
    "CFC-113",
    "CFC-114",
    "CFC-115",
    "HCFC-22",
    "HCFC-141b",
    "HCFC-142b",
    "CCl4",
    "CHCl3",
    "CH2Cl2",
    "CH3Cl",
    "CH3CCl3",
    "CH3Br",
    "Halon-1211",
    "Halon-1301",
    "Halon-2402",
    "CF4",
    "C2F6",
    "C3F8",
    "c-C4F8",
    "C4F10",
    "C5F12",
    "C6F14",
    "C7F16",
    "C8F18",
    "NF3",
    "SF6",
    "SO2F2",
    "HFC-125",
    "HFC-134a",
    "HFC-143a",
    "HFC-152a",
    "HFC-227ea",
    "HFC-23",
    "HFC-236fa",
    "HFC-245fa",
    "HFC-32",
    "HFC-365mfc",
    "HFC-4310mee",
]:
    fill(f_without.species_configs["forcing_scale"], df_configs["scale minorGHG"].values.squeeze(), specie=specie)

# ozone
for specie in ["CH4", "N2O", "Equivalent effective stratospheric chlorine", "CO", "VOC", "NOx"]:
    fill(f_without.species_configs["ozone_radiative_efficiency"], df_configs[f"o3 {specie}"], specie=specie)

# initial condition of CO2 concentration (but not baseline for forcing calculations)
fill(f_without.species_configs["baseline_concentration"], df_configs["co2_concentration_1750"].values.squeeze(), specie="CO2")

# initial conditions
initialise(f_without.concentration, f_without.species_configs["baseline_concentration"])
initialise(f_without.forcing, 0)
initialise(f_without.temperature, 0)
initialise(f_without.cumulative_emissions, 0)
initialise(f_without.airborne_emissions, 0)

### Do the run

In [ ]:
f_without.run()

## Run FaIR with extra HFC

We'll call this instance `f_with`. The `read_properties()` line points to the CSV file that includes HFC-152. We need to define `species_with` in advance, as it is no longer default.

In [ ]:
f_with = FAIR(ch4_method="Thornhill2021")
f_with.define_time(1750, 2300, 1)
f_with.define_scenarios(scenarios_with)
f_with.define_configs(valid_all)
species_with = species_without + ['HFC-152']  # just the normal set plus HFC-152
species_with, properties_with = read_properties(filename='../data/species_configs_properties_with.csv', species=species_with)
f_with.define_species(species_with, properties_with)
f_with.allocate()

In [ ]:
f_with.emissions

### Put in emissions from ssp245 + HFC-152

As before, but add in emissions from HFC-152. Maybe a 10 Gt pulse!

In [ ]:
gfed_sectors = [
    "Emissions|NOx|MAGICC AFOLU|Agricultural Waste Burning",
    "Emissions|NOx|MAGICC AFOLU|Forest Burning",
    "Emissions|NOx|MAGICC AFOLU|Grassland Burning",
    "Emissions|NOx|MAGICC AFOLU|Peat Burning",
]

for scenario in scenarios_with:
    f_with.emissions.loc[dict(specie="NOx", scenario=scenario)] = (
        df_emis.loc[
            (df_emis["Scenario"] == 'ssp245')
            & (df_emis["Region"] == "World")
            & (df_emis["Variable"].isin(gfed_sectors)),
            "1750":"2500",
        ]
        .interpolate(axis=1)
        .values.squeeze()
        .sum(axis=0)
        * 46.006
        / 30.006
        + df_emis.loc[
            (df_emis["Scenario"] == 'ssp245')
            & (df_emis["Region"] == "World")
            & (df_emis["Variable"] == "Emissions|NOx|MAGICC AFOLU|Agriculture"),
            "1750":"2500",
        ]
        .interpolate(axis=1)
        .values.squeeze()
        + df_emis.loc[
            (df_emis["Scenario"] == 'ssp245')
            & (df_emis["Region"] == "World")
            & (df_emis["Variable"] == "Emissions|NOx|MAGICC Fossil and Industrial"),
            "1750":"2500",
        ]
        .interpolate(axis=1)
        .values.squeeze()
    )[:550][:, None]

Now fill in the other emitted species.

In [ ]:
for scenario in scenarios_with:
    for specie in species_mapping:
        f_with.emissions.loc[dict(specie=specie, scenario=scenario)] = (
            df_emis.loc[
                (df_emis["Scenario"] == 'ssp245')
                & (df_emis["Region"] == "World")
                & (df_emis["Variable"] == species_mapping[specie]),
                "1750":"2500",
            ]
            .interpolate(axis=1)
            .values.squeeze()
            * unit_convert[specie]
        )[:550][:, None]

introduce a 10 Gt pulse in 2020. Minor GHGs are in units of kt, so this is quite large

In [ ]:
f_with.emissions.loc[dict(specie='HFC-152', scenario='ssp245_hfc152')] = 0
f_with.emissions.loc[dict(specie='HFC-152', scenario='ssp245_hfc152', timepoints=2020.5)] = 1e7

In [ ]:
f_with.emissions

### Put in solar and volcanic forcing

In [ ]:
fill(
    f_with.forcing,
    volcanic_forcing[:, None, None] * df_configs["scale Volcanic"].values.squeeze(),
    specie="Volcanic",
)
fill(
    f_with.forcing,
    solar_forcing[:, None, None] * df_configs["solar_amplitude"].values.squeeze()
    + trend_shape[:, None, None] * df_configs["solar_trend"].values.squeeze(),
    specie="Solar",
)

### Monte Carlo climate configs

In [ ]:
fill(f_with.climate_configs["ocean_heat_capacity"], df_configs.loc[:, "c1":"c3"].values)
fill(
    f_with.climate_configs["ocean_heat_transfer"],
    df_configs.loc[:, "kappa1":"kappa3"].values,
)
fill(f_with.climate_configs["deep_ocean_efficacy"], df_configs["epsilon"].values.squeeze())
fill(f_with.climate_configs["gamma_autocorrelation"], df_configs["gamma"].values.squeeze())
fill(f_with.climate_configs["sigma_eta"], df_configs["sigma_eta"].values.squeeze())
fill(f_with.climate_configs["sigma_xi"], df_configs["sigma_xi"].values.squeeze())
fill(f_with.climate_configs["seed"], df_configs["seed"])
fill(f_with.climate_configs["stochastic_run"], True)
fill(f_with.climate_configs["use_seed"], True)
fill(f_with.climate_configs["forcing_4co2"], df_configs["F_4xCO2"])

### Monte Carlo species configs

In the first line we need to point to the filename providing the configs. After this, we manually overrides what we loaded up in the CSV.

In [ ]:
f_with.fill_species_configs('../data/species_configs_properties_with.csv')

# carbon cycle
fill(f_with.species_configs["iirf_0"], df_configs["r0"].values.squeeze(), specie="CO2")
fill(f_with.species_configs["iirf_airborne"], df_configs["rA"].values.squeeze(), specie="CO2")
fill(f_with.species_configs["iirf_uptake"], df_configs["rU"].values.squeeze(), specie="CO2")
fill(f_with.species_configs["iirf_temperature"], df_configs["rT"].values.squeeze(), specie="CO2")

# aerosol indirect
fill(f_with.species_configs["aci_scale"], df_configs["beta"].values.squeeze())
fill(f_with.species_configs["aci_shape"], df_configs["shape Sulfur"].values.squeeze(), specie="Sulfur")
fill(f_with.species_configs["aci_shape"], df_configs["shape BC"].values.squeeze(), specie="BC")
fill(f_with.species_configs["aci_shape"], df_configs["shape OC"].values.squeeze(), specie="OC")

# aerosol direct
for specie in ["BC", "CH4", "N2O", "NH3", "NOx", "OC", "Sulfur", "VOC", "Equivalent effective stratospheric chlorine"]:
    fill(f_with.species_configs["erfari_radiative_efficiency"], df_configs[f"ari {specie}"], specie=specie)

# forcing scaling
for specie in ["CO2", "CH4", "N2O", "Stratospheric water vapour", "Contrails", "Light absorbing particles on snow and ice", "Land use"]:
    fill(f_with.species_configs["forcing_scale"], df_configs[f"scale {specie}"].values.squeeze(), specie=specie)

for specie in [
    "CFC-11",
    "CFC-12",
    "CFC-113",
    "CFC-114",
    "CFC-115",
    "HCFC-22",
    "HCFC-141b",
    "HCFC-142b",
    "CCl4",
    "CHCl3",
    "CH2Cl2",
    "CH3Cl",
    "CH3CCl3",
    "CH3Br",
    "Halon-1211",
    "Halon-1301",
    "Halon-2402",
    "CF4",
    "C2F6",
    "C3F8",
    "c-C4F8",
    "C4F10",
    "C5F12",
    "C6F14",
    "C7F16",
    "C8F18",
    "NF3",
    "SF6",
    "SO2F2",
    "HFC-125",
    "HFC-134a",
    "HFC-143a",
    "HFC-152a",
    "HFC-227ea",
    "HFC-23",
    "HFC-236fa",
    "HFC-245fa",
    "HFC-32",
    "HFC-365mfc",
    "HFC-4310mee",
]:
    fill(f_with.species_configs["forcing_scale"], df_configs["scale minorGHG"].values.squeeze(), specie=specie)

# ozone
for specie in ["CH4", "N2O", "Equivalent effective stratospheric chlorine", "CO", "VOC", "NOx"]:
    fill(f_with.species_configs["ozone_radiative_efficiency"], df_configs[f"o3 {specie}"], specie=specie)

# initial condition of CO2 concentration (but not baseline for forcing calculations)
fill(f_with.species_configs["baseline_concentration"], df_configs["co2_concentration_1750"].values.squeeze(), specie="CO2")

# initial conditions
initialise(f_with.concentration, f_with.species_configs["baseline_concentration"])
initialise(f_with.forcing, 0)
initialise(f_with.temperature, 0)
initialise(f_with.cumulative_emissions, 0)
initialise(f_with.airborne_emissions, 0)

## Do the run

In [ ]:
f_with.run()

## Basic analysis

In [ ]:
pl.plot(f_without.timebounds, f_without.temperature.loc[dict(layer=0)].median(dim='config'), label='ssp245')
pl.plot(f_with.timebounds, f_with.temperature.loc[dict(layer=0)].median(dim='config'), label='10Gt pulse HFC-152')
pl.ylabel('K relative to pre-industrial')
pl.title('Temperature anomaly')
pl.legend();

In [ ]:
pl.plot(
    range(2020, 2300), 
    f_with.temperature.loc[
        dict(layer=0, timebounds=range(2020, 2300))
    ].median(dim='config').values - 
    f_without.temperature.loc[
        dict(layer=0, timebounds=range(2020, 2300))
    ].median(dim='config').values
)
pl.ylabel('K since 2020')
pl.title('Temperature difference')

In [ ]:
pl.plot(
    range(2020, 2041), 
    f_with.temperature.loc[
        dict(layer=0, timebounds=range(2020, 2041))
    ].median(dim='config').values - 
    f_without.temperature.loc[
        dict(layer=0, timebounds=range(2020, 2041))
    ].median(dim='config').values
)
pl.ylabel('K since 2020')
pl.title('Temperature difference')

In [ ]:
pl.plot(
    range(2020, 2041), 
    f_with.concentration.loc[
        dict(specie='HFC-152', timebounds=range(2020, 2041))
    ].median(dim='config').values
)
pl.ylabel('ppt')
pl.title('HFC-152 concentration')

In [ ]:
pl.plot(
    range(2020, 2041), 
    f_with.forcing.loc[
        dict(specie='HFC-152', timebounds=range(2020, 2041))
    ].median(dim='config').values
)
pl.ylabel('W m$^{-2}$')
pl.title('HFC-152 effective radiative forcing')